In [15]:
# Import data analysis and visualisation packages.
import numpy as np
import pandas as pd
import stan as ps
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import arviz as az
import os
from patsy import dmatrix
from linearmodels.iv import IV2SLS

# Importing nest_asyncio is only necessary to run pystan in Jupyter Notebooks.
import nest_asyncio
nest_asyncio.apply()

#Specify the number of chains to the number of availible cpu's.
n_chains = os.cpu_count()
n_samples = 1000
#Convert to int so Stan will not crash below.
n_warmup = int(n_samples/2)
#Specify step size.
stepS = .8

In [19]:
def read_data(file): 
    return pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/"  + file)

card = read_data("card.dta")
card = card[['lwage', 'exper','black', 'south', 'married', 'smsa','educ', 'nearc4']]
cardNNA = card.dropna(axis=0)

In [3]:
IVMod = '''
functions
data{
int<lower = 1> N;
int K;
int K1;
int K2;
vector[K] y[N];
matrix[N, K1] X1;
matrix[N, K2] X2;
}
parameters{
// Regression parameters
vector[K1] beta1;  
vector[K2] beta2;  
vector<lower =0>[K] sigma;
// Correlation matrix
corr_matrix[K] rho;

}
transformed parameters{
// generate mu for each linear model
vector[N] mu1 = X1 * beta1;
vector[N] mu2 = X2 * beta2;
// generate a tempary matrix varaible
matrix[N, K] mu_temp = append_col(mu1, mu2);
// Convert that matrix into vectors for use by mulit-normal dist
// below in the likelihood
row_vector[K] mu[N];
for (n in 1:N)
    mu[n] = mu_temp[n];
}

model{
// Priors
sigma ~ exponential(1);

// Covariance matrix
matrix[K,K] Sigma;
// Uniform prior for correlation parameters
rho ~ lkj_corr(1);
Sigma = quad_form_diag(rho,sigma);

// linear model priors
beta1 ~ std_normal();
beta2 ~ std_normal();

// Likelihood
y ~ multi_normal(mu, Sigma);
}
'''

In [4]:
x1 = np.asarray(dmatrix('~1 + exper + black + south + married + smsa', data = cardNNA))
x2 = np.asarray(dmatrix(' ~ nearc4', data = cardNNA))
y = np.asarray(cardNNA[['lwage', 'educ']])

In [5]:
d ={'N': len(cardNNA),
    'y': y,
    'K': np.shape(y)[1],
    'K1': np.shape(x1)[1],
    'K2': np.shape(x2)[1],
    'X1': x1,
    'X2': x2
}

In [21]:
sm = ps.build(IVMod, data = d)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_al91jhcb/model_zjbvkmvq.stan', line 7, column 0: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_al91jhcb/model_zjbvkmvq.stan', line 25, column 0: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_al91jhcb/model_zjbvkmvq.stan', line 35, column 0: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.


In [22]:
fit = sm.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS)

Sampling:   0%
Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   0% (5/12000)
Sampling:   0% (6/12000)
Sampling:   0% (7/12000)
Sampling:   0% (8/12000)
Sampling:   1% (107/12000)
Sampling:   2% (206/12000)
Sampling:   3% (305/12000)
Sampling:   3% (404/12000)
Sampling:   4% (503/12000)
Sampling:   5% (602/12000)
Sampling:   6% (701/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2501/12000)
Sampling:  22% (2601/12000)
Sampling:  23% (2701/12000)
Sampling:  23% (2801/12000)
Sampling: 

  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because 

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35,

  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because 

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,2] = 1.68409e+151, but Covariance matrix[2,1] = 1.68409e+151 (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 42, column 0 to column 28)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: multi_normal_lpdf: Covariance matrix is not symmetric. Covariance matrix[1,2] = 3.90127e+37, but Covariance matrix[2,1] = 3.90127e+37 (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.st

  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because 

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.001027 seconds
  1000 transitions using 10 leapfrog steps per transition would take 10.27 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan_atcv0g6p/model_zjbvkmvq.stan', line 35, column 0 to column 18)
  If this warning occurs sporadically, such as for h

In [26]:
az.summary(fit, var_names = ['beta1', 'beta2', 'rho'])

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta1[0],6.005,0.037,5.937,6.074,0.000,0.000,5695.0,5388.0,1.0
beta1[1],0.040,0.003,0.034,0.046,0.000,0.000,6561.0,5754.0,1.0
beta1[2],-0.133,0.033,-0.193,-0.067,0.000,0.000,7529.0,5748.0,1.0
beta1[3],-0.096,0.021,-0.135,-0.057,0.000,0.000,7505.0,5465.0,1.0
beta1[4],-0.034,0.005,-0.043,-0.025,0.000,0.000,9485.0,6381.0,1.0
beta1[5],0.178,0.022,0.137,0.219,0.000,0.000,7407.0,5653.0,1.0
beta2[0],13.694,0.099,13.507,13.881,0.001,0.001,5765.0,5083.0,1.0
beta2[1],0.549,0.115,0.338,0.770,0.002,0.001,5724.0,5269.0,1.0
"rho[0, 0]",1.000,0.000,1.000,1.000,0.000,0.000,8000.0,8000.0,NaN
"rho[0, 1]",0.400,0.025,0.352,0.446,0.000,0.000,6568.0,5564.0,1.0


In [16]:
iv_reg = IV2SLS.from_formula("lwage ~ 1 + exper + black + south + married + smsa + [educ ~ nearc4 ]", card).fit()
iv_reg.su

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/linearmodels/iv/model.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                  lwage   R-squared:                      0.2513
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2498
No. Observations:                3003   F-statistic:                    892.71
Date:                Thu, Jul 13 2023   P-value (F-stat)                0.0000
Time:                        15:30:46   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.1625     0.8349     4.9857     0.0000      2.5262      5.7988
exper          0.0556     0.0199     2.7980     0.0051      0.0166      0.0945
black         -0.1157     0.0496    -2.3343     0.0196     -0.2128     -0.0186
south         -0.1132     0.0229    -4.9314     0.0000     -0.1581     -0.0682
married       -0.0320     0.0051    -6.3037     0.0000     -0.0419     -0.0220
smsa           0.1477     0.0303     4.8721     0.0000      0.0883      0.2071
educ           0.1242     0.0492     2.5258     0.0115      0.0278      0.2205
==============================================================================

Endogenous: educ
Instruments: nearc4
Robust Covariance (Heteroskedastic)
Debiased: False
"""